# NewV3 Student Fairness NetArch1

In [ ]:
!pip install ucimlrepo
!pip install fairlearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.quantization import quantize_dynamic, QConfig, prepare_qat, convert
from torch.quantization.fake_quantize import FakeQuantize
from torch.quantization.observer import MovingAverageMinMaxObserver
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from ucimlrepo import fetch_ucirepo
from fairlearn.metrics import MetricFrame
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    true_positive_rate,
    false_positive_rate
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 2.7 MB/s eta 0:00:00


##Code Persistence

In [ ]:
def save_model(model, model_path):
  torch.save(model.state_dict(), model_path)
"""
def save_values_to_txt(data, filename):
  with open(filename, 'a') as f:
      for entry in data:
          for key, value in entry.items():
              f.write(f'{key}: {value}\n')
          f.write('\n')
  print(f"Data appended to {filename}")
"""
def save_values_to_txt(data, filename):
    if isinstance(data, dict):
        data = [data]
    with open(filename, 'a') as f:
        for entry in data:
            for key, value in entry.items():
                f.write(f'{key}: {value}\n')
            f.write('\n')
    print(f"Data appended to {filename}")

In [ ]:
random.seed(42)
np.random.seed(42)

### Data Loading

In [ ]:
def load_your_dataset():
    # fetch dataset
    student = fetch_ucirepo(id=320)

    X = student.data.features
    y = student.data.targets.G3
    y = (y > 10).astype(int)
    le = LabelEncoder()
    y = le.fit_transform(y.values.ravel())  # e.g. 0 or 1

    X_np = X.values
    y_np = y

    X_dummies = pd.get_dummies(X, drop_first=True)

    scaler = StandardScaler()
    X_np = scaler.fit_transform(X_dummies)


    A_np = X["sex"].values
    print(A_np.size)
    print(X["sex"].value_counts())

    return X_np, y_np, A_np

X_np, y_np, A_np = load_your_dataset()

649
sex
F    383
M    266
Name: count, dtype: int64


## Fairness Evaluation

* Statistical Parity Difference: Closer to 0 is better. 0 means no bias.
* Average Odds Difference: 0 is ideal, indicating no discrimination.
* Disparate Impact: Aim for 1. Values close to 1 indicate fairness.
* Theil Index: Lower is better. 0 means perfect equality.

In [ ]:
def calculate_statistical_parity_difference(y_true, y_pred, A_test):
    sr = lambda y_true, y_pred: selection_rate(y_true, y_pred)
    mfm = MetricFrame(metrics=sr, y_true=y_test, y_pred=y_pred, sensitive_features=A_test)
    return mfm.difference(method='between_groups')

def calculate_average_odds_difference(y_true, y_pred, A_test):
    unique_labels = np.unique(y_true)
    if len(unique_labels) == 2:
        pos_label = unique_labels[1]
    else:
        raise ValueError("y_true should have exactly two unique values for binary classification")

    tpr = lambda y_true, y_pred: true_positive_rate(y_true, y_pred, pos_label=pos_label)
    fpr = lambda y_true, y_pred: false_positive_rate(y_true, y_pred, pos_label=pos_label)
    average_odds = lambda y_true, y_pred: (tpr(y_true, y_pred) + fpr(y_true, y_pred)) / 2

    mf = MetricFrame(metrics=average_odds,
                     y_true=y_true,
                     y_pred=y_pred,
                     sensitive_features=A_test)
    return mf.difference(method='between_groups')

def calculate_disparate_impact(y_true, y_pred, A_test):
    sr = lambda y_true, y_pred: selection_rate(y_true, y_pred)
    mf = MetricFrame(metrics=sr, y_true=y_true, y_pred=y_pred, sensitive_features=A_test)
    return mf.ratio(method='between_groups')

def calculate_theil_index(y_true, y_pred):
    actual_pos = np.mean(y_true == 1)
    pred_pos = np.mean(y_pred == 1)

    epsilon = 1e-10

    actual_entropy = -(actual_pos * np.log2(actual_pos + epsilon) + (1 - actual_pos) * np.log2(1 - actual_pos + epsilon))
    pred_entropy = -(pred_pos * np.log2(pred_pos + epsilon) + (1 - pred_pos) * np.log2(1 - pred_pos + epsilon))

    theil_index = pred_entropy - actual_entropy

    #mf = MetricFrame(metrics=sr, y_true=y_true, y_pred=y_pred, sensitive_features=A_test)
    #mf.difference(method='between_groups')
    return theil_index
    """
    actual_pos = np.mean(y_true == 1)
    pred_pos = np.mean(y_pred == 1)

    epsilon = 1e-10

    # KL Divergence Calculation (non-negative)
    actual_entropy = -(actual_pos * np.log2(actual_pos + epsilon) + (1 - actual_pos) * np.log2(1 - actual_pos + epsilon))
    pred_entropy = -(pred_pos * np.log2(pred_pos + epsilon) + (1 - pred_pos) * np.log2(1 - pred_pos + epsilon))

    theil_index = np.abs(pred_entropy - actual_entropy)  # Use absolute difference to avoid negatives
    return theil_index
    """

##Model Architecture

In [ ]:
n_input = X_np.shape[1]

class NetArch1(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(n_input, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

        self.layernorm1 = nn.LayerNorm(128)
        self.layernorm2 = nn.LayerNorm(64)

    def forward(self, x):
        x = self.layernorm1(F.relu(self.fc1(x)))
        x = self.layernorm2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

class NetArch2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_input, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

class NetArch3(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_input, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 2)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

## QAT Compatible Model Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.quantization

n_input = X_np.shape[1]

class QATArch1(nn.Module):
    def __init__(self):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.fc1 = nn.Linear(n_input, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.layernorm1 = nn.LayerNorm(128)
        self.layernorm2 = nn.LayerNorm(64)
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.layernorm1(F.relu(self.fc1(x)))
        x = self.layernorm2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

class QATArch2(nn.Module):
    def __init__(self):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.fc1 = nn.Linear(n_input, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 2)
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

class QATArch3(nn.Module):
    def __init__(self):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.fc1 = nn.Linear(n_input, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 2)
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = self.fc4(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def dynamic_compress_model(model_path):
    model = NetArch1()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    compressed_model = quantize_dynamic(
        model=model,
        qconfig_spec={torch.nn.Linear},
        dtype=torch.qint8
    )
    return compressed_model

def prepare_model_for_qat(model):
    model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    model = torch.quantization.prepare_qat(model)
    return model

def convert_qat_model(model):
    model = torch.quantization.convert(model.eval())
    return model


def get_k_fold_split(X, y,A, k, num_splits=5):
    indices = np.arange(len(X))
    fold_size = len(X) // num_splits

    start = k * fold_size
    end = start + fold_size if k != num_splits - 1 else len(X)

    test_idx = indices[start:end]
    train_idx = np.concatenate([indices[:start], indices[end:]])

    return X[train_idx], y[train_idx], A[train_idx], X[test_idx], y[test_idx], A[test_idx]


def train_baseline_model(X_train, y_train, fold, num_epochs=10, batch_size=64):
  model = NetArch1()
  optimizer = optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.NLLLoss()

  dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
  train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  model.train()
  for epoch in range(num_epochs):
      for batch_X, batch_y in train_loader:
          optimizer.zero_grad()
          outputs = model(batch_X)
          loss = criterion(outputs, batch_y)
          loss.backward()
          optimizer.step()

  model.eval()
  y_true, y_pred = [], []
  with torch.no_grad():
      for batch_X, batch_y in train_loader:
          outputs = model(batch_X)
          _, predicted = torch.max(outputs, 1)
          y_true.extend(batch_y.numpy())
          y_pred.extend(predicted.numpy())

  f1 = f1_score(y_true, y_pred)
  print(f"Fold {fold + 1}: Train F1 Score = {f1:.4f}")


  model_path = f"model_fold{fold}.pkl"
  save_model(model, model_path)

  return model

def train_qat_model(X_train, y_train, fold, num_epochs=10, batch_size=64):
    model = QATArch1()
    model = prepare_model_for_qat(model)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.NLLLoss()

    dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for batch_X, batch_y in train_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_X.size(0)

    model.eval()
    quantized_model = convert_qat_model(model)

    y_true, y_pred = [], []
    with torch.no_grad():
        for batch_X, batch_y in train_loader:
            outputs = quantized_model(batch_X)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(batch_y.numpy())
            y_pred.extend(predicted.numpy())

    f1 = f1_score(y_true, y_pred)
    print(f"Fold {fold + 1}: Train F1 Score (QAT) = {f1:.4f}")

    model_path = f"qat_model_fold{fold}.pkl"
    save_model(quantized_model, model_path)

    return quantized_model


def evaluate_baseline_model(model, X_test, y_test, A_test):
    output = model.forward(torch.FloatTensor(X_test))
    _, y_pred = torch.max(output, dim=1)

    y_pred = pd.Series(y_pred.detach().numpy())
    y_test = pd.Series(y_test)

    mf1 = MetricFrame(metrics=f1_score, y_true=y_test, y_pred=y_pred, sensitive_features=A_test)

    baseline_metrics = {
        "Baseline Statistical Parity Difference": round(calculate_statistical_parity_difference(y_test, y_pred, A_test), 3),
        "Baseline Average Odds Difference": round(calculate_average_odds_difference(y_test, y_pred, A_test), 3),
        "Baseline Disparate Impact": round(calculate_disparate_impact(y_test, y_pred, A_test), 3),
        "Baseline Theil Index": round(calculate_theil_index(y_test, y_pred),3),
        "Baseline F1 Score": round(f1_score(y_test, y_pred),3)
    }

    return baseline_metrics

def evaluate_qat_model(model, X_test, y_test, A_test):
    X_test = X_test.astype(np.float32)

    nan_mask = np.isnan(X_test)
    if nan_mask.any():
        col_means = np.nanmean(X_test, axis=0)
        X_test[nan_mask] = col_means[nan_mask[0]]

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

    model.eval()

    with torch.no_grad():
        output = model(X_test_tensor)
        _, y_pred = torch.max(output, dim=1)

    y_pred_np = y_pred.cpu().numpy()

    qat_metrics = {
        "QAT Statistical Parity Difference": round(calculate_statistical_parity_difference(y_test, y_pred_np, A_test), 3),
        "QAT Average Odds Difference": round(calculate_average_odds_difference(y_test, y_pred_np, A_test), 3),
        "QAT Disparate Impact": round(calculate_disparate_impact(y_test, y_pred_np, A_test), 3),
        "QAT Theil Index": round(calculate_theil_index(y_test, y_pred_np), 3),
        "QAT F1 Score": round(f1_score(y_test, y_pred_np), 3),
        "F1 Score": round(f1_score(y_test, y_pred_np), 3)
    }

    return qat_metrics

def evaluate_compressed_model(model, X_test, y_test, A_test):
    output = model(torch.FloatTensor(X_test))
    _, y_pred = torch.max(output, dim=1)

    y_pred = pd.Series(y_pred.detach().numpy())
    y_test = pd.Series(y_test)

    mf = MetricFrame(metrics=f1_score, y_true=y_test, y_pred=y_pred, sensitive_features=A_test)

    compressed_metrics = {
        "Compressed Statistical Parity Difference": round(calculate_statistical_parity_difference(y_test, y_pred, A_test), 3),
        "Compressed Average Odds Difference": round(calculate_average_odds_difference(y_test, y_pred, A_test), 3),
        "Compressed Disparate Impact": round(calculate_disparate_impact(y_test, y_pred, A_test), 3),
        "Compressed Theil Index": round(calculate_theil_index(y_test, y_pred), 3),
        "Compressed F1 Score": round(f1_score(y_test, y_pred), 3),
    }

    return compressed_metrics


### k‑Fold Cross Validation


In [ ]:

X, y, A = load_your_dataset()
num_splits = 10

baseline_fold_metrics = []
compressed_fold_metrics = []
qat_fold_metrics = []

for k in range(num_splits):
    print(f"\nProcessing fold {k + 1}/{num_splits}")

    X_train, y_train, A_train, X_test, y_test, A_test = get_k_fold_split(X, y, A, k, num_splits)

    baseline_model = train_baseline_model(X_train, y_train, k)
    baseline_eval = evaluate_baseline_model(baseline_model, X_test, y_test, A_test)
    model_path = f"model_fold{k}.pkl"

    compressed_model = dynamic_compress_model(model_path)
    compressed_eval = evaluate_compressed_model(compressed_model, X_test, y_test, A_test)

    qat_model = train_qat_model(X_train, y_train, k)
    qat_eval = evaluate_qat_model(qat_model, X_test, y_test, A_test)

    baseline_fold_metrics.append(baseline_eval)
    compressed_fold_metrics.append(compressed_eval)
    qat_fold_metrics.append(qat_eval)

    print(f"Baseline Eval: {baseline_eval}")
    print(f"Compressed Eval: {compressed_eval}")
    print(f"QAT Eval: {qat_eval}")

baseline_df = pd.DataFrame(baseline_fold_metrics)
compressed_df = pd.DataFrame(compressed_fold_metrics)
qat_df = pd.DataFrame(qat_fold_metrics)


baseline_df.to_csv("baseline_metrics.csv", index=True, float_format="%.3f")
compressed_df.to_csv("compressed_metrics.csv", index=True, float_format="%.3f")
qat_df.to_csv("qat_metrics.csv", index=True, float_format="%.3f")



Processing fold 1/10
Fold 1: Train F1 Score = 0.9899


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Fold 1: Train F1 Score (QAT) = 0.9726
Baseline Eval: {'Baseline Statistical Parity Difference': np.float64(0.062), 'Baseline Average Odds Difference': np.float64(0.076), 'Baseline Disparate Impact': np.float64(0.933), 'Baseline Theil Index': np.float64(-0.049), 'Baseline F1 Score': 0.922}
Compressed Eval: {'Compressed Statistical Parity Difference': np.float64(0.062), 'Compressed Average Odds Difference': np.float64(0.076), 'Compressed Disparate Impact': np.float64(0.933), 'Compressed Theil Index': np.float64(-0.049), 'Compressed F1 Score': 0.922}
QAT Eval: {'QAT Statistical Parity Difference': np.float64(0.062), 'QAT Average Odds Difference': np.float64(0.076), 'QAT Disparate Impact': np.float64(0.933), 'QAT Theil Index': np.float64(-0.049), 'QAT F1 Score': 0.922, 'F1 Score': 0.922}

Processing fold 2/10
Fold 2: Train F1 Score = 0.9851


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Fold 2: Train F1 Score (QAT) = 0.9701
Baseline Eval: {'Baseline Statistical Parity Difference': np.float64(0.066), 'Baseline Average Odds Difference': np.float64(0.146), 'Baseline Disparate Impact': np.float64(0.931), 'Baseline Theil Index': np.float64(-0.359), 'Baseline F1 Score': 0.893}
Compressed Eval: {'Compressed Statistical Parity Difference': np.float64(0.066), 'Compressed Average Odds Difference': np.float64(0.146), 'Compressed Disparate Impact': np.float64(0.931), 'Compressed Theil Index': np.float64(-0.359), 'Compressed F1 Score': 0.893}
QAT Eval: {'QAT Statistical Parity Difference': np.float64(0.036), 'QAT Average Odds Difference': np.float64(0.046), 'QAT Disparate Impact': np.float64(0.961), 'QAT Theil Index': np.float64(-0.301), 'QAT F1 Score': 0.901, 'F1 Score': 0.901}

Processing fold 3/10
Fold 3: Train F1 Score = 0.9807


...

Fold 8: Train F1 Score (QAT) = 0.9764
Baseline Eval: {'Baseline Statistical Parity Difference': np.float64(0.223), 'Baseline Average Odds Difference': np.float64(0.243), 'Baseline Disparate Impact': np.float64(0.544), 'Baseline Theil Index': np.float64(0.0), 'Baseline F1 Score': 0.719}
Compressed Eval: {'Compressed Statistical Parity Difference': np.float64(0.223), 'Compressed Average Odds Difference': np.float64(0.243), 'Compressed Disparate Impact': np.float64(0.544), 'Compressed Theil Index': np.float64(0.0), 'Compressed F1 Score': 0.719}
QAT Eval: {'QAT Statistical Parity Difference': np.float64(0.044), 'QAT Average Odds Difference': np.float64(0.099), 'QAT Disparate Impact': np.float64(0.915), 'QAT Theil Index': np.float64(0.011), 'QAT F1 Score': 0.765, 'F1 Score': 0.765}

Processing fold 9/10
Fold 9: Train F1 Score = 0.9849


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Fold 9: Train F1 Score (QAT) = 0.9873
Baseline Eval: {'Baseline Statistical Parity Difference': np.float64(0.263), 'Baseline Average Odds Difference': np.float64(0.204), 'Baseline Disparate Impact': np.float64(0.642), 'Baseline Theil Index': np.float64(0.009), 'Baseline F1 Score': 0.635}
Compressed Eval: {'Compressed Statistical Parity Difference': np.float64(0.263), 'Compressed Average Odds Difference': np.float64(0.204), 'Compressed Disparate Impact': np.float64(0.642), 'Compressed Theil Index': np.float64(0.009), 'Compressed F1 Score': 0.635}
QAT Eval: {'QAT Statistical Parity Difference': np.float64(0.255), 'QAT Average Odds Difference': np.float64(0.204), 'QAT Disparate Impact': np.float64(0.618), 'QAT Theil Index': np.float64(0.032), 'QAT F1 Score': 0.61, 'F1 Score': 0.61}

Processing fold 10/10
Fold 10: Train F1 Score = 0.9855


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Fold 10: Train F1 Score (QAT) = 0.9857
Baseline Eval: {'Baseline Statistical Parity Difference': np.float64(0.321), 'Baseline Average Odds Difference': np.float64(0.339), 'Baseline Disparate Impact': np.float64(0.51), 'Baseline Theil Index': np.float64(-0.008), 'Baseline F1 Score': 0.75}
Compressed Eval: {'Compressed Statistical Parity Difference': np.float64(0.273), 'Compressed Average Odds Difference': np.float64(0.283), 'Compressed Disparate Impact': np.float64(0.583), 'Compressed Theil Index': np.float64(-0.013), 'Compressed F1 Score': 0.741}
QAT Eval: {'QAT Statistical Parity Difference': np.float64(0.206), 'QAT Average Odds Difference': np.float64(0.197), 'QAT Disparate Impact': np.float64(0.675), 'QAT Theil Index': np.float64(-0.013), 'QAT F1 Score': 0.667, 'F1 Score': 0.667}


In [ ]:
print(baseline_df)
baseline_df.to_csv("baseline_metrics.csv", index=True, float_format="%.3f")

   Baseline Statistical Parity Difference  Baseline Average Odds Difference  \
0                                   0.062                             0.076   
1                                   0.066                             0.146   
2                                   0.267                             0.106   
3                                   0.003                             0.033   
4                                   0.012                             0.118   
5                                   0.002                             0.167   
6                                   0.166                             0.022   
7                                   0.223                             0.243   
8                                   0.263                             0.204   
9                                   0.321                             0.339   

   Baseline Disparate Impact  Baseline Theil Index  Baseline F1 Score  
0                      0.933                -0.049        

In [ ]:
!zip -r colab_files.zip /content
from google.colab import files
files.download('colab_files.zip')

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.05.09/ (stored 0%)
  adding: content/.config/logs/2025.05.09/13.40.57.224020.log (deflated 92%)
  adding: content/.config/logs/2025.05.09/13.41.27.718573.log (deflated 58%)
  adding: content/.config/logs/2025.05.09/13.41.26.495445.log (deflated 87%)
  adding: content/.config/logs/2025.05.09/13.41.18.092482.log (deflated 58%)
  adding: content/.config/logs/2025.05.09/13.41.36.263788.log (deflated 57%)
  adding: content/.config/logs/2025.05.09/13.41.36.930373.log (deflated 57%)
  adding: content/.config/hidden_gcloud_config_universe_descript

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Friedman test